In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ========================================
# 1. LEER LOS ARCHIVOS
# ========================================

# Test data (CSV)
test_df = pd.read_csv(r'data/raw/test.csv')

In [36]:
# ========================================
# VERIFICACIÓN DE VALORES FALTANTES
# ========================================

print("\n VERIFICACIÓN DE VALORES FALTANTES")
print("="*45)

# TEST DATA
print(" TEST DATA:")
missing_test = test_df.isnull().sum()
print(missing_test)
if missing_test.sum() == 0:
    print("No hay valores faltantes en test data")
else:
    print("Valores faltantes encontrados:")
    print(f"  Total celdas faltantes: {missing_test.sum()}")
    print(f"  Porcentaje: {(missing_test.sum() / test_df.size) * 100:.2f}%")


 VERIFICACIÓN DE VALORES FALTANTES
 TEST DATA:
management    0
group         0
htls          0
client_id     0
dtype: int64
No hay valores faltantes en test data


In [37]:
# ========================================
# VERIFICACIÓN DE DUPLICADOS
# ========================================

print("\n VERIFICACIÓN DE DUPLICADOS")
print("="*40)

# TEST DATA
print(" TEST DATA:")
print(f"  - Total filas: {len(test_df)}")
print(f"  - client_id únicos: {test_df['client_id'].nunique()}")
print(f"  - Duplicados en client_id: {len(test_df) - test_df['client_id'].nunique()}")

if test_df['client_id'].duplicated().any():
    print(" DUPLICADOS ENCONTRADOS en test data:")
    duplicados_test = test_df[test_df['client_id'].duplicated(keep=False)]
    print(duplicados_test.sort_values('client_id'))
else:
    print("No hay duplicados en client_id (test data)")


 VERIFICACIÓN DE DUPLICADOS
 TEST DATA:
  - Total filas: 10000
  - client_id únicos: 10000
  - Duplicados en client_id: 0
No hay duplicados en client_id (test data)


In [38]:
# VERIFICACIÓN DE TIPOS DE DATOS
# ========================================

print("\n VERIFICACIÓN DE TIPOS DE DATOS")
print("="*40)

print(" TEST DATA:")
print(test_df.dtypes)


 VERIFICACIÓN DE TIPOS DE DATOS
 TEST DATA:
management     object
group          object
htls          float64
client_id       int64
dtype: object


In [39]:
# ========================================
print("VERIFICACIÓN DE VALORES ÚNICOS")
print("="*40)

# Verificar si hay valores extraños en variables categóricas
print("TEST DATA - Variables categóricas:")
print(f"  management: {sorted(test_df['management'].unique())}")
print(f"  group: {sorted(test_df['group'].unique())}")



VERIFICACIÓN DE VALORES ÚNICOS
TEST DATA - Variables categóricas:
  management: ['A', 'B', 'C']
  group: ['Control', 'Test']


In [40]:
# ========================================
# RESUMEN ESTADÍSTICO DE HTLS
# ========================================

print(f"\n TEST DATA - RESUMEN ESTADÍSTICO HTLS:")
print("="*45)

# Estadísticas descriptivas básicas
htls_stats = test_df['htls'].describe()
print(" Estadísticas descriptivas:")
print(f"  Count:    {htls_stats['count']:>10.0f}")
print(f"  Mean:     {htls_stats['mean']:>10.2f}")
print(f"  Std:      {htls_stats['std']:>10.2f}")
print(f"  Min:      {htls_stats['min']:>10.2f}")
print(f"  25%:      {htls_stats['25%']:>10.2f}")
print(f"  50%:      {htls_stats['50%']:>10.2f}")
print(f"  75%:      {htls_stats['75%']:>10.2f}")
print(f"  Max:      {htls_stats['max']:>10.2f}")

# Estadísticas adicionales
print(f"\n Estadísticas adicionales:")
print(f"  Skewness:     {test_df['htls'].skew():>8.3f}")
print(f"  Kurtosis:     {test_df['htls'].kurtosis():>8.3f}")
print(f"  Range:        {htls_stats['max'] - htls_stats['min']:>8.2f}")
print(f"  IQR:          {htls_stats['75%'] - htls_stats['25%']:>8.2f}")
print(f"  CV (%):       {(htls_stats['std']/htls_stats['mean'])*100:>8.1f}")

# Verificaciones adicionales
print(f"\n Verificaciones adicionales:")
print(f"  Valores negativos:  {(test_df['htls'] < 0).sum():>6}")
print(f"  Valores cero:       {(test_df['htls'] == 0).sum():>6}")
print(f"  Valores > Q3+1.5*IQR: {(test_df['htls'] > htls_stats['75%'] + 1.5*(htls_stats['75%']-htls_stats['25%'])).sum():>4}")

# Percentiles adicionales 
percentiles = [0.01, 0.05, 0.10, 0.90, 0.95, 0.99]
print(f"\n Percentiles extremos:")
for p in percentiles:
    value = test_df['htls'].quantile(p)
    print(f"  {p*100:4.1f}%:      {value:>10.2f}")


 TEST DATA - RESUMEN ESTADÍSTICO HTLS:
 Estadísticas descriptivas:
  Count:         10000
  Mean:          62.67
  Std:          143.94
  Min:            0.02
  25%:           14.07
  50%:           30.04
  75%:           62.13
  Max:         6850.77

 Estadísticas adicionales:
  Skewness:       16.204
  Kurtosis:      556.926
  Range:         6850.75
  IQR:             48.06
  CV (%):          229.7

 Verificaciones adicionales:
  Valores negativos:       0
  Valores cero:            0
  Valores > Q3+1.5*IQR:  921

 Percentiles extremos:
   1.0%:            1.12
   5.0%:            3.56
  10.0%:            6.23
  90.0%:          125.49
  95.0%:          202.43
  99.0%:          584.56


*Resumen del EDA*
- La variable HTLS presenta una distribución extremadamente asimétrica (skewness = 16.2) con alta variabilidad (CV = 229.7%) y 921 outliers, violando severamente los supuestos de normalidad y homoscedasticidad requeridos para ANOVA. La ausencia de valores negativos o cero permite implementar transformación logarítmica como estrategia correctiva obligatoria. Decisión: Proceder con log(HTLS) para el análisis, validando posteriormente los supuestos mediante pruebas de normalidad y homoscedasticidad